# Connect to Snowflake

In [1]:
from dotenv import load_dotenv
load_dotenv()     # loads keys into os.environ so the rest of your code sees them

True

In [2]:
# authenticate into Snowflake
from snowflake.snowpark import Session
import os
connection_parameters = {
    "account": os.getenv('SNOWFLAKE_ACCOUNT'),
    "user": os.getenv('SNOWFLAKE_USER'),
    "password": os.getenv('SNOWFLAKE_PASSWORD'),
    "role": os.getenv('SNOWFLAKE_ROLE'),
    "warehouse": os.getenv('SNOWFLAKE_WAREHOUSE'),
    "database": os.getenv('SNOWFLAKE_DATABASE'),
    "schema": os.getenv('SNOWFLAKE_SCHEMA')
}
session = Session.builder.configs(connection_parameters).create()

In [3]:
# check connection has been successful
print("Session Current Account:", session.get_current_account())

Session Current Account: "WEVIRIP-NA38028"


# Data Exploration

In [4]:
# load tables into session
purchase_history = session.table('PURCHASE_HISTORY')
campaign_info = session.table('CAMPAIGN_INFO')
complain_info = session.table('COMPLAINT_INFO')

In [5]:
# show first 5 rows of purchase table
purchase_history.show(n=5)

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ID"  |"YEAR_BIRTH"  |"EDUCATION"  |"MARITAL_STATUS"  |"INCOME"  |"KIDHOME"  |"TEENHOME"  |"DT_CUSTOMER"  |"RECENCY"  |"MNTWINES"  |"MNTFRUITS"  |"MNTMEATPRODUCTS"  |"MNTFISHPRODUCTS"  |"MNTSWEETPRODUCTS"  |"MNTGOLDPRODS"  |"NUMDEALSPURCHASES"  |"NUMWEBPURCHASES"  |"NUMCATALOGPURCHASES"  |"NUMSTOREPURCHASES"  |"NUMWEBVISITSMONTH"  |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [6]:
# use the collect() method to display data (more suited to gather and retrieve data)
purchase_history.collect()

[Row(ID=5524, YEAR_BIRTH=1957, EDUCATION='Graduation', MARITAL_STATUS='Single', INCOME=58138, KIDHOME=0, TEENHOME=0, DT_CUSTOMER=datetime.date(2012, 9, 4), RECENCY=58, MNTWINES=635, MNTFRUITS=88, MNTMEATPRODUCTS=546, MNTFISHPRODUCTS=172, MNTSWEETPRODUCTS=88, MNTGOLDPRODS=88, NUMDEALSPURCHASES=3, NUMWEBPURCHASES=8, NUMCATALOGPURCHASES=10, NUMSTOREPURCHASES=4, NUMWEBVISITSMONTH=7),
 Row(ID=2174, YEAR_BIRTH=1954, EDUCATION='Graduation', MARITAL_STATUS='Single', INCOME=46344, KIDHOME=1, TEENHOME=1, DT_CUSTOMER=datetime.date(2014, 3, 8), RECENCY=38, MNTWINES=11, MNTFRUITS=1, MNTMEATPRODUCTS=6, MNTFISHPRODUCTS=2, MNTSWEETPRODUCTS=1, MNTGOLDPRODS=6, NUMDEALSPURCHASES=2, NUMWEBPURCHASES=1, NUMCATALOGPURCHASES=1, NUMSTOREPURCHASES=2, NUMWEBVISITSMONTH=5),
 Row(ID=4141, YEAR_BIRTH=1965, EDUCATION='Graduation', MARITAL_STATUS='Together', INCOME=71613, KIDHOME=0, TEENHOME=0, DT_CUSTOMER=datetime.date(2013, 8, 21), RECENCY=26, MNTWINES=426, MNTFRUITS=49, MNTMEATPRODUCTS=127, MNTFISHPRODUCTS=111, MN

In [7]:
# get count of rows
purchase_history.count()

2000

In [8]:
# show the columns
purchase_history.columns

['ID',
 'YEAR_BIRTH',
 'EDUCATION',
 'MARITAL_STATUS',
 'INCOME',
 'KIDHOME',
 'TEENHOME',
 'DT_CUSTOMER',
 'RECENCY',
 'MNTWINES',
 'MNTFRUITS',
 'MNTMEATPRODUCTS',
 'MNTFISHPRODUCTS',
 'MNTSWEETPRODUCTS',
 'MNTGOLDPRODS',
 'NUMDEALSPURCHASES',
 'NUMWEBPURCHASES',
 'NUMCATALOGPURCHASES',
 'NUMSTOREPURCHASES',
 'NUMWEBVISITSMONTH']

In [9]:
# filter specific rows
from snowflake.snowpark.functions import col
purchase_history.filter(col('id') == 1).show()

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ID"  |"YEAR_BIRTH"  |"EDUCATION"  |"MARITAL_STATUS"  |"INCOME"  |"KIDHOME"  |"TEENHOME"  |"DT_CUSTOMER"  |"RECENCY"  |"MNTWINES"  |"MNTFRUITS"  |"MNTMEATPRODUCTS"  |"MNTFISHPRODUCTS"  |"MNTSWEETPRODUCTS"  |"MNTGOLDPRODS"  |"NUMDEALSPURCHASES"  |"NUMWEBPURCHASES"  |"NUMCATALOGPURCHASES"  |"NUMSTOREPURCHASES"  |"NUMWEBVISITSMONTH"  |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [10]:
# multiple filters
purchase_history.filter((col('MARITAL_STATUS') == 'Married') & (col('KIDHOME') == 1)).show() 

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ID"   |"YEAR_BIRTH"  |"EDUCATION"  |"MARITAL_STATUS"  |"INCOME"  |"KIDHOME"  |"TEENHOME"  |"DT_CUSTOMER"  |"RECENCY"  |"MNTWINES"  |"MNTFRUITS"  |"MNTMEATPRODUCTS"  |"MNTFISHPRODUCTS"  |"MNTSWEETPRODUCTS"  |"MNTGOLDPRODS"  |"NUMDEALSPURCHASES"  |"NUMWEBPURCHASES"  |"NUMCATALOGPURCHASES"  |"NUMSTOREPURCHASES"  |"NUMWEBVISITSMONTH"  |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [11]:
# filter range of values
purchase_history.filter((col('YEAR_BIRTH') >= 1964) & (col('YEAR_BIRTH') <= 1980)).show()

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ID"  |"YEAR_BIRTH"  |"EDUCATION"  |"MARITAL_STATUS"  |"INCOME"  |"KIDHOME"  |"TEENHOME"  |"DT_CUSTOMER"  |"RECENCY"  |"MNTWINES"  |"MNTFRUITS"  |"MNTMEATPRODUCTS"  |"MNTFISHPRODUCTS"  |"MNTSWEETPRODUCTS"  |"MNTGOLDPRODS"  |"NUMDEALSPURCHASES"  |"NUMWEBPURCHASES"  |"NUMCATALOGPURCHASES"  |"NUMSTOREPURCHASES"  |"NUMWEBVISITSMONTH"  |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [12]:
# select a subset of columns
purchase_history.select(col('ID'), col('YEAR_BIRTH'), col('EDUCATION')).show()

-------------------------------------
|"ID"  |"YEAR_BIRTH"  |"EDUCATION"  |
-------------------------------------
|5524  |1957          |Graduation   |
|2174  |1954          |Graduation   |
|4141  |1965          |Graduation   |
|6182  |1984          |Graduation   |
|5324  |1981          |PhD          |
|7446  |1967          |Master       |
|965   |1971          |Graduation   |
|6177  |1985          |PhD          |
|4855  |1974          |PhD          |
|5899  |1950          |PhD          |
-------------------------------------



# Data Transformation

In [14]:
# inner join b/w purchase_history and campaign_info
purchase_campaign = purchase_history.join(
    campaign_info,
    purchase_history.ID == campaign_info.ID,
    lsuffix='_left', rsuffix='_right'
)

# drop extra ID column
purchase_campaign = purchase_campaign.drop('ID_RIGHT')

# show dataframe
purchase_campaign.show()

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ID_LEFT"  |"YEAR_BIRTH"  |"EDUCATION"  |"MARITAL_STATUS"  |"INCOME"  |"KIDHOME"  |"TEENHOME"  |"DT_CUSTOMER"  |"RECENCY"  |"MNTWINES"  |"MNTFRUITS"  |"MNTMEATPRODUCTS"  |"MNTFISHPRODUCTS"  |"MNTSWEETPRODUCTS"  |"MNTGOLDPRODS"  |"NUMDEALSPURCHASES"  |"NUMWEBPURCHASES"  |"NUMCATALOGPURCHASES"  |"NUMSTOREPURCHASES"  |"NUMWEBVISITSMONTH"  |"ACCEPTEDCMP1"  |"ACCEPTEDCMP2"  |"ACCEPTEDCMP3"  |"ACCEPTEDCMP4"  |"ACCEPTEDCMP5"  |"RESPONSE"  |
--------------------------------------------------------------------------------------------------------------------------

In [15]:
# combine with complaint information
final_combined = purchase_campaign.join(
    complain_info,
    purchase_campaign['ID_LEFT'] == complain_info.ID
)
final_combined = final_combined.drop('ID_LEFT')
final_combined.show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"YEAR_BIRTH"  |"EDUCATION"  |"MARITAL_STATUS"  |"INCOME"  |"KIDHOME"  |"TEENHOME"  |"DT_CUSTOMER"  |"RECENCY"  |"MNTWINES"  |"MNTFRUITS"  |"MNTMEATPRODUCTS"  |"MNTFISHPRODUCTS"  |"MNTSWEETPRODUCTS"  |"MNTGOLDPRODS"  |"NUMDEALSPURCHASES"  |"NUMWEBPURCHASES"  |"NUMCATALOGPURCHASES"  |"NUMSTOREPURCHASES"  |"NUMWEBVISITSMONTH"  |"ACCEPTEDCMP1"  |"ACCEPTEDCMP2"  |"ACCEPTEDCMP3"  |"ACCEPTEDCMP4"  |"ACCEPTEDCMP5"  |"RESPONSE"  |"ID"  |"COMPLAIN"  |"Z_COSTCONTACT"  |"Z_REVENUE"  |
------------------------------------------

In [16]:
# write combined data to table
final_combined.write.save_as_table('MARKETING_DATA')

In [18]:
# ingest marketing_additional csv file
session.file.put('./datasets/marketing_additional.csv', 'MY_STAGE')

import snowflake.snowpark.types as T

marketing_additional_schema = T.StructType([T.StructField("ID", T.IntegerType()),
        T.StructField("Year_Birth", T.IntegerType()),T.StructField("Education", T.StringType()),
        T.StructField("Marital_Status", T.StringType()),T.StructField("Income", T.IntegerType()),
        T.StructField("Kidhome", T.IntegerType()),T.StructField("Teenhome", T.IntegerType()),
        T.StructField("Dt_Customer", T.DateType()),T.StructField("Recency", T.IntegerType()),
        T.StructField("MntWines", T.IntegerType()),T.StructField("MntFruits", T.IntegerType()),
        T.StructField("MntMeatProducts", T.IntegerType()),T.StructField("MntFishProducts", T.IntegerType()),
        T.StructField("MntSweetProducts", T.IntegerType()),T.StructField("MntGoldProds", T.IntegerType()),
        T.StructField("NumDealsPurchases", T.IntegerType()),T.StructField("NumWebPurchases", T.IntegerType()),
        T.StructField("NumCatalogPurchases", T.IntegerType()),T.StructField("NumStorePurchases", T.IntegerType()),
        T.StructField("NumWebVisitsMonth", T.IntegerType()),T.StructField("AcceptedCmp3", T.IntegerType()),
        T.StructField("AcceptedCmp4", T.IntegerType()),T.StructField("AcceptedCmp5", T.IntegerType()),
        T.StructField("AcceptedCmp1", T.IntegerType()),T.StructField("AcceptedCmp2", T.IntegerType()),
        T.StructField("Complain", T.IntegerType()),T.StructField("Z_CostContact", T.IntegerType()),
        T.StructField("Z_Revenue", T.IntegerType()),T.StructField("Response", T.DecimalType())
])

session.sql("TRUNCATE TABLE IF EXISTS MARKETING_ADDITIONAL").collect()

marketing_additional = session.read\
        .option("FIELD_DELIMITER", ',')\
        .option("SKIP_HEADER", 1)\
        .option("ON_ERROR", "CONTINUE")\
        .schema(marketing_additional_schema).csv("@MY_Stage/marketing_additional.csv.gz")\
        .copy_into_table("MARKETING_ADDITIONAL")

session.table("MARKETING_ADDITIONAL").show()

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ID"   |"YEAR_BIRTH"  |"EDUCATION"  |"MARITAL_STATUS"  |"INCOME"  |"KIDHOME"  |"TEENHOME"  |"DT_CUSTOMER"  |"RECENCY"  |"MNTWINES"  |"MNTFRUITS"  |"MNTMEATPRODUCTS"  |"MNTFISHPRODUCTS"  |"MNTSWEETPRODUCTS"  |"MNTGOLDPRODS"  |"NUMDEALSPURCHASES"  |"NUMWEBPURCHASES"  |"NUMCATALOGPURCHASES"  |"NUMSTOREPURCHASES"  |"NUMWEBVISITSMONTH"  |"ACCEPTEDCMP3"  |"ACCEPTEDCMP4"  |"ACCEPTEDCMP5"  |"ACCEPTEDCMP1"  |"ACCEPTEDCMP2"  |"COMPLAIN"  |"Z_COSTCONTACT"  |"Z_REVENUE"  |"RESPONSE"  |
----------------------------------------

In [19]:
# load marketing additional table
marketing_additional = session.table('MARKETING_ADDITIONAL')
marketing_additional.show()

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ID"   |"YEAR_BIRTH"  |"EDUCATION"  |"MARITAL_STATUS"  |"INCOME"  |"KIDHOME"  |"TEENHOME"  |"DT_CUSTOMER"  |"RECENCY"  |"MNTWINES"  |"MNTFRUITS"  |"MNTMEATPRODUCTS"  |"MNTFISHPRODUCTS"  |"MNTSWEETPRODUCTS"  |"MNTGOLDPRODS"  |"NUMDEALSPURCHASES"  |"NUMWEBPURCHASES"  |"NUMCATALOGPURCHASES"  |"NUMSTOREPURCHASES"  |"NUMWEBVISITSMONTH"  |"ACCEPTEDCMP3"  |"ACCEPTEDCMP4"  |"ACCEPTEDCMP5"  |"ACCEPTEDCMP1"  |"ACCEPTEDCMP2"  |"COMPLAIN"  |"Z_COSTCONTACT"  |"Z_REVENUE"  |"RESPONSE"  |
----------------------------------------

In [20]:
# print number of rows
print('No of rows in MARKETING_ADDITIONAL table: ', marketing_additional.count())
print('No of rows in PURCHASE_HISTORY table: ', final_combined.count())

No of rows in MARKETING_ADDITIONAL table:  240
No of rows in PURCHASE_HISTORY table:  2000


In [21]:
# append the data
final_appended = final_combined.union_by_name(marketing_additional)

In [22]:
# print number of rows
print('No of rows in UPDATED table: ', final_appended.count())
final_appended.show()

No of rows in UPDATED table:  2240
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"YEAR_BIRTH"  |"EDUCATION"  |"MARITAL_STATUS"  |"INCOME"  |"KIDHOME"  |"TEENHOME"  |"DT_CUSTOMER"  |"RECENCY"  |"MNTWINES"  |"MNTFRUITS"  |"MNTMEATPRODUCTS"  |"MNTFISHPRODUCTS"  |"MNTSWEETPRODUCTS"  |"MNTGOLDPRODS"  |"NUMDEALSPURCHASES"  |"NUMWEBPURCHASES"  |"NUMCATALOGPURCHASES"  |"NUMSTOREPURCHASES"  |"NUMWEBVISITSMONTH"  |"ACCEPTEDCMP1"  |"ACCEPTEDCMP2"  |"ACCEPTEDCMP3"  |"ACCEPTEDCMP4"  |"ACCEPTEDCMP5"  |"RESPONSE"  |"ID"  |"COMPLAIN"  |"Z_COSTCONTACT"  |"Z_REVENUE"  |
-------

In [23]:
# write to table
final_appended.write.save_as_table('MARKETING_FINAL')

# Close Snowflake Session

In [24]:
# always close a session
session.close()